<a href="https://colab.research.google.com/github/julialromero/regional-representations-graph-model/blob/master/dataset/earth_engine/download_landsat_imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install pydrive

In [1]:
# ! pip install geemap

In [1]:
import sys
import os
import geemap
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

In [2]:
import ee
# ee.Authenticate()
ee.Initialize()

In [202]:
def makeRectangle(xy, xDistance, yDistance, proj, geoid):
    x = xy[0]
    y = xy[1]
    geometry = ee.Geometry.Rectangle(
                  [x - xDistance, 
                   y - yDistance,
                   x + xDistance, 
                   y + yDistance], proj, False)
    
    return ee.Feature(geometry).set({'GEOID': geoid})


xDistance = 2434
yDistance = 2434
threshold_size = ee.Number(xDistance*yDistance)


In [213]:
feat = tracts.filter(ee.Filter.eq('GEOID', geoid)).first()
roi = feat.geometry()

In [214]:
roi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Element.geometry",
    "arguments": {
      "feature": {
        "functionInvocationValue": {
          "functionName": "Collection.first",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection",
                      "arguments": {
                        "features": {
                          "arrayValue": {
                            "values": [
                              {
                                "functionInvocationValue": {
                                  "functionName": "Element.set",
                                  "arguments": {
                                    "key": {
                                      "constantValue": "GEOID"
                                    },
                                    "object": {
                                      "functionInvocationValue": {
                                        "functionName": "Feature",
                                        "arguments": {
                                          "geometry": {
                                            "functionInvocationValue": {
                                              "functionName": "GeometryConstructors.Rectangle",
                                              "arguments": {
                                                "coordinates": {
                                                  "constantValue": [
                                                    -5581295.035963135,
                                                    9006984.96075102,
                                                    -5576427.035963135,
                                                    9011852.96075102
                                                  ]
                                                },
                                                "crs": {
                                                  "functionInvocationValue": {
                                                    "functionName": "Projection",
                                                    "arguments": {
                                                      "crs": {
                                                        "constantValue": "EPSG:32632"
                                                      }
                                                    }
                                                  }
                                                },
                                                "geodesic": {
                                                  "constantValue": false
                                                }
                                              }
                                            }
                                          }
                                        }
                                      }
                                    },
                                    "value": {
                                      "constantValue": "09009350400"
                                    }
                                  }
                                }
                              },
                              {
                                "functionInvocationValue": {
                                  "functionName": "Element.set",
                                  "arguments": {
                                    "key": {
                                      "constantValue": "GEOID"
                                    },
                                    "object": {
                                      "functionInvocationValue": {
                                        "functionName": "Feature",
                                        "

In [215]:
def get_img(geoid):
    feat = tracts.filter(ee.Filter.eq('GEOID', geoid)).first()
    roi = feat.geometry()
    
    collection = (
      ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')  # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1
      .filterBounds(roi)
      .filter(ee.Filter.contains('.geo', roi))
      .filter(ee.Filter.calendarRange(6, 10, 'month'))
      .filterDate('2016-01-01', '2023-01-01')
      .sort('CLOUD_COVER')
    )
    top = collection.limit(1)

    # take the median pixel value; clip to the census geometry 
    # composite = top.median().clip(roi)
#     print(roi)
    img = top.first().clip(roi).reproject(crs='EPSG:4326')
    return img#, geoid
    

def createDir(dirPath):
  try:
    os.makedirs(dirPath)
  except:
    pass

def init_dir(subdir, fn):
  if os.path.exists(subdir):
      if fn in os.listdir(subdir):
#         print('pass')
        return True
     
  if not os.path.exists(subdir):
      print('creating directory')
      createDir(subdir)
      return True

In [216]:
tract_data_dir = r'../Tracts/nyc_metro_boundaries/nyc_metro_boundaries.shp'
temp_tract_data_dir = "../Tracts/nyc_metro_boundaries/temp.shp"
temp_envelope_data_dir = "../Tracts/nyc_metro_boundaries/temp_envelope.shp"

df = gpd.read_file(tract_data_dir)
df.rename(columns={'GEOID_TRAC': 'GEOID'}, inplace=True)  
# print(df.head())
shapefile = df[['GEOID', 'geometry']] 
# shapefile['geometry'] = shapefile['geometry'].envelope

In [217]:
earth_engine_filename = 'least_cloudy_m6_m10_rectangle_fixedlargesize.tif'
fn = earth_engine_filename

In [218]:
# clip or not clip the images to census polygon
earth_engine_clip_image_to_geometry = False

# Run to download Landsat Images

In [219]:
# https://stackoverflow.com/questions/60748584/create-ee-geometry-rectangle-centered-at-coordinate-point-and-specify-size-in-pi

In [220]:
def calculate_centroid_meters(sub):
    sub['centroid'] = sub.to_crs('EPSG:32632').geometry.centroid
    centroids = sub[['GEOID', 'centroid']]
    centroids.set_geometry("centroid", inplace=True, crs='EPSG:32632')
    centroids['x'] = centroids.centroid.x
    centroids['y'] = centroids.centroid.y

    d = dict(zip(centroids['GEOID'], zip(centroids['x'], centroids['y'])))
    return d

In [221]:
# ee_rectangle_geometries = ee.List(list(rectangle_geometries))
# ee_rectangle_geometries
#     rect_feat_collection = ee.FeatureCollection(ee_rectangle_geometries)

In [231]:
save_dir = 'download_landsat_images'
count = 0
for interval in range(200, shapefile.shape[0], 50): 
    try:
        sub = shapefile.iloc[interval:interval+50]
    except:
        sub = shapefile.iloc[interval:-1]
        
    # Convert tract polygon to the envelope polygon
    if earth_engine_clip_image_to_geometry == False:
        sub['geometry'] = sub['geometry'].envelope

    sub.to_file(temp_envelope_data_dir)
    tracts = geemap.shp_to_ee(temp_envelope_data_dir)
    
    ###### for fixed rectangle size #######
    d = calculate_centroid_meters(sub)
    rectangle_geometries = sub['GEOID'].apply(lambda x: makeRectangle(d[x], xDistance, yDistance, 'EPSG:32632', x))
    ee_rectangle_geometries = ee.List(list(rectangle_geometries))
    rect_feat_collection = ee.FeatureCollection(ee_rectangle_geometries)
    
    # export and import to change projection
    out_shp = "tester.shp"
    geemap.ee_export_vector(rect_feat_collection, out_shp, verbose=True)
    rect_feat_collection = geemap.shp_to_ee(out_shp)
    
    
    geoids = tracts.aggregate_array('GEOID')
    tracts=rect_feat_collection
    
    geoids = tracts.aggregate_array('GEOID')
    result = geoids.map(get_img)
    
    # download loop
    for i in range(0, sub.shape[0]):
        geoid = shapefile.iloc[interval+i]['GEOID']
        
        subdir = f"{save_dir}/{geoid}"
        filename = f"{subdir}/" + fn
        
#         if os.path.exists(filename):
#             continue
            
        init_dir(subdir, filename)
         
      
        img = ee.Image(result.get(i))
        img = img.select(['B2', 'B3', 'B4', 'B5'])
                
        geemap.ee_export_image(
                img, filename=filename, file_per_band=False, 
            )
#        
    print(f'-----DONE {interval} - {interval + sub.shape[0]}')
print(f'Number of images with scale not at 30 m resolution: {count}')
    

/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/tester.shp
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001044200/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001073000/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001073200/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001073300/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Pleas

Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001210300/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001080900/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001022201/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001022202/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001022101

Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001035300/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001073600/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001050100/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001060500/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001100300/least_cloudy_m6_m1

/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

...

Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/tester.shp
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34039037602/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34039038201/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34039035200/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34039031700/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Pleas

Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34039035300/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34039035500/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34039035600/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34039035800/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34039035900/least_cloudy_m6_m10_rectangle_fixe

Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34019011306/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34019011305/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34019010202/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34019011003/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34019011004

/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/tester.shp
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34035054302/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34035054301/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34035052604/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34035052605/least_cloudy_m6_m10_rectangle_fixedlargesize.tif
-----DONE 5500 - 5504
Nu

In [ ]:
# # plot all
# Map = geemap.Map()
# Map.setCenter(-74.0060, 40.7128, 8)
# visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
# Map.addLayer(top, visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
# Map

# Run for rectangle census tract images

In [34]:
def get_regions_to_retry():
    l = []
    counter=0
    for root, dirs, files in os.walk("download_landsat_images/"):
        if '.ipynb_checkpoints' in root:
            continue
        if fn not in files:
            l.append(root)
            continue
        else:
            counter+=1
        
            
    l.remove('download_landsat_images/')
    al = len(os.listdir("download_landsat_images/"))
    print(f'Num folders total: {al}')
    print(f'Num target files total: {counter}')
    print()
    l = [item[-11:] for item in l]
    return l
    
r = get_regions_to_retry()
print(len(r))
# for geoid in [r]:
#     shapefile.loc[shapefile.GEOID == geoid].to_file(temp_tract_data_dir)
#     tracts = geemap.shp_to_ee(temp_tract_data_dir)
    
#     # Convert tract polygon to the envelope polygon
#     envelope_shp = shapefile.loc[shapefile.GEOID == geoid]
#     envelope_shp['geometry'] = envelope_shp['geometry'].envelope
#     envelope_shp.to_file(temp_envelope_data_dir)
#     envelope_tracts = geemap.shp_to_ee(temp_envelope_data_dir)
    
#     img = get_img(geoid, envelope_tracts)
#     save_dir = 'download_landsat_images'
#     subdir = f"{save_dir}/{geoid}"
#     filename = f"{subdir}/" + fn

# #     img = ee.Image(result.get(i))
#     img = img.select(['B2', 'B3', 'B4', 'B5'])

#     try:
#         if img.projection().nominalScale().getInfo() != 30:
#             count +=1
#             continue
#     except:
#         print('--->Error getting projection. Passing...')

#     geemap.ee_export_image(
#             img, filename=filename, file_per_band=False)


#     print('--\n')

Num folders total: 8232
Num target files total: 5503

2728


In [39]:
df.shape[0]

5504

In [ ]:
# # plot all
# Map = geemap.Map()
# Map.setCenter(-74.0060, 40.7128, 8)
# visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
# Map.addLayer(collection.first(), visParams, 'Landsat8')
# Map

# Test package



In [22]:
cld

In [23]:
img

In [21]:
ee.Initialize()

aoi = ee.Geometry(Point([-48.4, -23]))

clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterDate('2019-11-25', '2019-11-28').filterBounds(aoi)

# img = s2.first()
cld = clouds.first()

Map = geemap.Map(center=(-23, -48.4), zoom=9)
Map.addLayer(cld, {'min': 0, 'max': 100, 'palette': ['purple', 'yellow']}, 'Cloud Probability', True, 1)
Map

Map(center=[-23, -48.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

## Scratchwork

In [ ]:
import tqdm


stats = pd.DataFrame(columns = [
    'geoid',
    'max',
 'mean',
 'min',
 'sample_sd',
 'sample_var',
 'sum',
 'sum_sq',
 'total_count',
 'total_sd',
 'total_var',
 'valid_count',
 'weight_sum',
 'weighted_sum'])

img_list = ee.List([])


def save_images(geoid):
    print(geoid_list)
    for geoid in geoid_list:
        print(geoid)
    save_dir = 'download_landsat_images/'
    subdir = f"{save_dir}{geoid}"


    if os.path.exists(subdir):
        if len(os.listdir(subdir)) >= 10:
#               print('pass')
          return True
        else:
#               print(f'Redoing: {subdir}')
            pass

    if not os.path.exists(subdir):
#             print('creating directory')
        createDir(subdir)

    
    geemap.ee_export_image(img, filename=f'{subdir}/least_CC.tif', region=roi, scale=90)

    # for image collection stats 
#         temp = pd.DataFrame(data=img.toArray().getInfo()).T
#         temp.columns=temp.keys().getInfo()
#         temp['geoid'] = geoid

#         stats = pd.concat([stats, temp], axis=0)
#         stats.to_csv('save_dir/_CC_stats.csv', index=False)


#         for img_idx in top.aggregate_array('system:index').getInfo():
#             img = ee.Image('LANDSAT/LC08/C02/T1_TOA/' + img_idx)
#             filename = f"{subdir}/{img_idx}.tif"

#             new_img = img.clip(roi)

#             geemap.ee_export_image(
#                 new_img, filename=filename, scale=90, region=roi, file_per_band=False
#             )

    return 

In [ ]:
# plot all
Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
Map.addLayer(collection.first(), visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
Map

In [ ]:
Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
Map.addLayer(new_img, visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
Map

In [ ]:
save_dir = 'download_landsat_images/'
def Count_files_in_subd():
    l = []
    
    for root, dirs, files in os.walk(save_dir):
        # print("{} in {}".format(len(files), root))
        l.append(len(files))
#         if len(files) == 0:
#           print(dirs)
    al = len(os.listdir(save_dir))
    print(f'Num folders total: {al}')
    print(f'Num folders with 0 files: {l.count(0)}')
    print(f'Num folders with >0 files: {al - l.count(0)}')
    print()
Count_files_in_subd()